## Import historical data to MongoDB

In [24]:
import csv
import logging
import pymongo
import os
from tqdm import tnrange, tqdm_notebook

logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

In [ ]:
client = pymongo.MongoClient('localhost', 27017)
db = client.tanken
preise = db.preise
stationen = db.stationen

In [1]:
def load_preis_csv(path):
    with open(path,"r") as file:
        return [row for row in csv.DictReader(file)]

In [27]:
daten_dir = "daten/"


if not os.path.exists(daten_dir):
    logging.error("Daten-Ordner nicht gefunden")
    exit()
else:
    if "preise" in os.listdir(daten_dir):
        for year in os.listdir(daten_dir+"preise/"):
            for month in os.listdir(daten_dir+"preise/"+year):
                for day in os.listdir(daten_dir+"preise/"+year+"/"+month):
                    _ = preise.insert_many(load_preis_csv(daten_dir+"preise/"+year+"/"+month+"/"+day))


MemoryError: 

In [28]:
print(preise)


MemoryError: 